In [1]:
from data_handlers import StockHistDataHandler,ComputeSuite
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings("ignore")
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import xarray as xr
data_handler = StockHistDataHandler()
from utilities.function_utilities import *

In [2]:
static = data_handler(sql = "select * from securities")
np.unique(static["sector"])

array(['Communication Services', 'Consumer Discretionary',
       'Consumer Staples', 'Energy', 'Financials', 'Health Care',
       'Industrials', 'Information Technology', 'Materials',
       'Real Estate', 'Utilities'], dtype=object)

In [3]:
tickers = data_handler(sql = "select ticker from securities where sector in ('Information Technology')").values.ravel()
ts_range = ["2010-01-05", "2020-01-20"]
selected_tickers = tickers
all_data = data_handler.get_data(ticker = selected_tickers, invest_horizen = ts_range)

In [4]:
start_trading = "2010-01-5"
returns = ComputeSuite.pct_return(all_data.loc[:,:,start_trading:])
returns.index = all_data.loc[:,:,start_trading:].ticker
returns.columns = np.array(all_data.loc[:,:,start_trading:]["eff_date"])
returns.loc["MARKET"] = returns.mean(axis=0)
# returns = returns.T

ValueError: Length mismatch: Expected axis has 2527 elements, new values have 74 elements

In [5]:

from strategy_base import strategy, _null_handle
class buy_low_strategy(strategy):

    @_null_handle
    def generate_signal(self, event_data):
        sig = np.zeros(event_data.shape[1])

        previous_market = np.nanmean(event_data.values[-2])
        previous_returns = event_data.values[-2]
        actual_returns = event_data.values[-1]
        mkt_condition = (previous_market > 0.00)
        stock_condition = (previous_returns < -0.02) & (event_data.values[0] < 0.0)
        sig[stock_condition & mkt_condition] = actual_returns[stock_condition & mkt_condition]
        return sig


In [ ]:
def rolling_apply_ext(window ,data, func, *args ,**kwargs):
    """
    pandas' implementation of rolling.apply is such a disappointment!
    :param window:
    :param data:
    :param func:
    :param args:
    :param kwargs:
    :return:
    """
    df_idx = data.index
    result_df = pd.DataFrame().reindex_like(data)

    for i in range(len(df_idx)):
        sub_df = data.loc[df_idx[ i -window +1]:df_idx[i]]
        result_df.loc[df_idx[i] ] =func(sub_df ,*args ,**kwargs)
    return result_df

In [ ]:

bl_Strategy = buy_low_strategy()
buy_returns = rolling_apply_ext(3, returns, bl_Strategy.generate_signal)

In [ ]:
fig = plt.figure(figsize=(12,3))
_ = sns.heatmap(np.clip(buy_returns,-0.06,0.06).T,cmap="PiYG").set(xticklabels=[])


In [ ]:
from scipy.stats import skew,kurtosis

In [ ]:
buy_returns[buy_returns==0]=np.nan
sns.kdeplot(np.nanmean(buy_returns,axis=1))
sns.kdeplot(returns["MARKET"].values)

print(f"buy:{100*np.nanmean(np.nanmean(buy_returns,axis=1)):.5f}")
print(f"mkt:{100*np.nanmean(returns['MARKET'].values):.5f}")